In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./part.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                resp = requests.get(input_.loc[a, 'Info'],
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text

                # = = = = = = = = = = = = = = =

                soup = BeautifulSoup(resp, 'lxml')
                html = etree.HTML(str(soup))
                
                # = = = = = = = = = = = = = = =
                
                list_tbody = html.xpath('//section[contains(@aria-label, "Components") and contains(@aria-label, "kit")]/table/tbody/tr/td/table/tbody[contains(@class, "listing-inner") and (contains(@class, "altrow-b-0") or contains(@class, "altrow-b-1"))]')

                # = = = = = = = = = = = = = = =
                
                list_quantity = [int(tbody.xpath('./tr/td[1]')[0].text.strip()) for tbody in list_tbody]
                
                # = = = = = = = = = = = = = = =

                list_note = [tbody.xpath('./tr/td[2]')[0].text.strip() for tbody in list_tbody]
                
                # = = = = = = = = = = = = = = =
                
                list_json = [json.loads(tbody.xpath('./tr/td[contains(@id, "listing_data_essential")]/@value')[0]) for tbody in list_tbody]

                # = = = = = = = = = = = = = = =
                
                list_type_code = [json_['parttype'].strip() for json_ in list_json]
                
                # = = = = = = = = = = = = = = =
                
                list_part_key = [json_['partkey'].strip() for json_ in list_json]
                
                # = = = = = = = = = = = = = = =
                
                list_json = [json.loads(tbody.xpath('./tr/input[contains(@id, "listing_data_supplemental")]/@value')[0]) for tbody in list_tbody]

                # = = = = = = = = = = = = = = =

                list_manufacturer = [json_['catalogname'].strip() for json_ in list_json]
                
                # = = = = = = = = = = = = = = =
                
                list_part_number = [json_['partnumber'].strip() for json_ in list_json]
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame({'Info': input_.loc[a, 'Info'],
                                        'Kit_No.': [i+1 for i in range(len(list_tbody))],
                                        'Kit_Quantity': list_quantity,
                                        'Kit_Note': list_note,
                                        'Kit_Manufacturer': list_manufacturer,
                                        'Kit_Type_Code': list_type_code,
                                        'Kit_Part_Key': list_part_key,
                                        'Kit_Part_Number': list_part_number})

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Info': input_.loc[a, 'Info']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Info'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(5):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Info', 'Kit_No.'], ascending=[True, True]).reset_index(drop=True)
output_correct.to_excel('./kit_part_number-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Info'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./kit_part_number_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：488

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8304976
[尝试次数：1] - [剩余数量：483] - [当前时间：16:13:58]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=11158045
[尝试次数：1] - [剩余数量：482] - [当前时间：16:13:58]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8304972
[尝试次数：1] - [剩余数量：481] - [当前时间：16:13:58]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8304968
[尝试次数：1] - [剩余数量：480] - [当前时间：16:13:59]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8304960
[尝试次数：1] - [剩余数量：479] - [当前时间：16:13:59]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10110000
[尝试次数：1] - [剩余数量：478] - [当前时间：16:14:00]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=9658984
[尝试次数：1] - [剩余数量：477] - [当前时间：16:14:00]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8304980
[尝试次数：1] - [剩余数量：476] - [当前时间：16:14:00]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8304984
[尝试次数：1] - [剩余数量：475] - [当前时间：16:14:00]

[ok] - https://ww

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=9722084
[尝试次数：2] - [剩余数量：407] - [当前时间：16:14:50]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=9854468
[尝试次数：1] - [剩余数量：406] - [当前时间：16:14:52]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=9854472
[尝试次数：1] - [剩余数量：405] - [当前时间：16:14:55]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=12804013
[尝试次数：7] - [剩余数量：404] - [当前时间：16:14:56]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=12804005
[尝试次数：27] - [剩余数量：403] - [当前时间：16:14:56]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305044
[尝试次数：7] - [剩余数量：402] - [当前时间：16:14:57]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10890268
[尝试次数：1] - [剩余数量：401] - [当前时间：16:14:58]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=9854480
[尝试次数：5] - [剩余数量：400] - [当前时间：16:14:59]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10890272
[尝试次数：5] - [剩余数量：399] - [当前时间：16:15:01]

[ok] - https://www.rock

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=5031150
[尝试次数：1] - [剩余数量：331] - [当前时间：16:15:56]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=7096416
[尝试次数：11] - [剩余数量：330] - [当前时间：16:15:57]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305208
[尝试次数：3] - [剩余数量：329] - [当前时间：16:15:57]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=5031153
[尝试次数：1] - [剩余数量：328] - [当前时间：16:15:58]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=5031147
[尝试次数：9] - [剩余数量：327] - [当前时间：16:15:58]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305216
[尝试次数：2] - [剩余数量：326] - [当前时间：16:15:59]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=12902177
[尝试次数：1] - [剩余数量：325] - [当前时间：16:15:59]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10890276
[尝试次数：4] - [剩余数量：324] - [当前时间：16:16:00]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305236
[尝试次数：2] - [剩余数量：323] - [当前时间：16:16:01]

[ok] - https://www.rockau

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305380
[尝试次数：1] - [剩余数量：255] - [当前时间：16:16:50]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=11158141
[尝试次数：2] - [剩余数量：254] - [当前时间：16:16:52]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305384
[尝试次数：6] - [剩余数量：253] - [当前时间：16:16:52]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305388
[尝试次数：7] - [剩余数量：252] - [当前时间：16:16:53]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305396
[尝试次数：3] - [剩余数量：251] - [当前时间：16:16:54]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=11492497
[尝试次数：1] - [剩余数量：250] - [当前时间：16:16:54]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305392
[尝试次数：1] - [剩余数量：249] - [当前时间：16:16:55]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10195072
[尝试次数：1] - [剩余数量：248] - [当前时间：16:16:55]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=11158149
[尝试次数：3] - [剩余数量：247] - [当前时间：16:16:57]

[ok] - https://www.rocka

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=15383101
[尝试次数：3] - [剩余数量：179] - [当前时间：16:18:12]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=12804065
[尝试次数：1] - [剩余数量：178] - [当前时间：16:18:13]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=15383097
[尝试次数：3] - [剩余数量：177] - [当前时间：16:18:14]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305584
[尝试次数：3] - [剩余数量：176] - [当前时间：16:18:16]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305580
[尝试次数：8] - [剩余数量：175] - [当前时间：16:18:16]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305588
[尝试次数：8] - [剩余数量：174] - [当前时间：16:18:20]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=5031156
[尝试次数：10] - [剩余数量：173] - [当前时间：16:18:20]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305576
[尝试次数：17] - [剩余数量：172] - [当前时间：16:18:21]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305592
[尝试次数：2] - [剩余数量：171] - [当前时间：16:18:21]

[ok] - https://www.rock

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8305800
[尝试次数：6] - [剩余数量：103] - [当前时间：16:19:07]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=12804081
[尝试次数：6] - [剩余数量：102] - [当前时间：16:19:08]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10890304
[尝试次数：2] - [剩余数量：101] - [当前时间：16:19:09]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10454536
[尝试次数：3] - [剩余数量：100] - [当前时间：16:19:09]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10890300
[尝试次数：5] - [剩余数量：99] - [当前时间：16:19:09]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=13570081
[尝试次数：4] - [剩余数量：98] - [当前时间：16:19:09]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=12804089
[尝试次数：2] - [剩余数量：97] - [当前时间：16:19:11]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=14047677
[尝试次数：9] - [剩余数量：96] - [当前时间：16:19:12]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=13570085
[尝试次数：5] - [剩余数量：95] - [当前时间：16:19:12]

[ok] - https://www.rockau

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=13572057
[尝试次数：1] - [剩余数量：26] - [当前时间：16:19:59]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=13572053
[尝试次数：4] - [剩余数量：25] - [当前时间：16:20:00]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=13572061
[尝试次数：2] - [剩余数量：24] - [当前时间：16:20:02]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=8716112
[尝试次数：7] - [剩余数量：23] - [当前时间：16:20:02]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=14047965
[尝试次数：5] - [剩余数量：22] - [当前时间：16:20:03]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=14047969
[尝试次数：2] - [剩余数量：21] - [当前时间：16:20:04]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=14047981
[尝试次数：1] - [剩余数量：20] - [当前时间：16:20:05]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=14047977
[尝试次数：3] - [剩余数量：19] - [当前时间：16:20:06]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=14047985
[尝试次数：2] - [剩余数量：18] - [当前时间：16:20:07]

[ok] - https://www.rockauto.c